In [29]:
import pandas as pd
import numpy as np
import seaborn as sn
import scipy.optimize as opt 
from sklearn import preprocessing 
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold 
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_curve, roc_auc_score, roc_curve 
from matplotlib import pyplot 
from sklearn.preprocessing import LabelEncoder

In [6]:
col_list = ["Date"," Time"," EpochTime"," LogOp"," Pid"," package"," freeMemory"," CpuTime"," ProcessCpuLoad"," CurrentThreadCpuTime"," CurrentThreaduserTime"," SystemLoadAverage"," UserId"," TimeDiff"," Error"," Message"," Outcome"]
df = pd.read_csv("final_data2.csv", usecols=col_list)

/Users/tanvi.kini/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (6,7,8,9,10,11,12,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [7]:
# Remove non-important columns
# Remove info and Warning logs
# Reverse the data set to get the logs at the end is when the outage occured

df=df.rename(columns=lambda x: x.strip())
df=df.drop(columns=['Date','Time','Pid','package','UserId','Error',"Message"])
df

,EpochTime,LogOp,freeMemory,CpuTime,ProcessCpuLoad,CurrentThreadCpuTime,CurrentThreaduserTime,SystemLoadAverage,TimeDiff,Outcome
0,1596127227063,INFO,643836128,65660059000,0.0625,5940593000,4914928000,3.66015625,0.0,30
1,1596127227067,INFO,643836128,65667444000,0.1332972582972583,5942996000,4915261000,3.66015625,4.0,0
2,1596127227067,INFO,643836128,65667509000,0.0646186440677966,5943055000,4915299000,3.66015625,0.0,0
3,1596127227067,INFO,643836128,65667551000,0.0625,5943095000,4915322000,3.66015625,0.0,0
4,1596127227067,INFO,643836128,65667599000,0.06380208333333333,5943143000,4915346000,3.66015625,0.0,0
...,...,...,...,...,...,...,...,...,...,...
198447,1596101934472,WARN,72796720,7030262000,0.0631313,3153912000,2829502000,3.00537,0.0,0
198448,1596101934473,INFO,72796720,7030421000,0.0625,3154073000,2829582000,3.00537,1.0,0
198449,1596101934473,WARN,72796720,7030421000,0.0625,3154073000,2829582000,3.00537,0.0,0
198450,1596101934473,INFO,72796720,7030593000,0.0621387,3154245000,2829675000,3.00537,0.0,0


In [8]:
df.replace(' ', np.nan, inplace= True)
# df=df[(df.EpochTime != 1596127449372)&(df.EpochTime != 1596127449376)&(df.EpochTime != 1596127449379)]
# df=df[(df.EpochTime <= 1596127216746)&(df.EpochTime >= 1596127218946)]

# df
df.dropna(inplace=True)

In [10]:
for col in df[df.columns[2:9]]:
    df[col]=pd.to_numeric(df[col])

In [11]:
for col in df[df.columns[2:8]]:
    df[col]=df[col].interpolate(method='pad')
# df.fillna(method='pad')
# dff.fillna(dff.mean())
# ts.interpolate()
# df.mean(1)

In [12]:
df.dtypes

EpochTime                  int64
LogOp                     object
freeMemory                 int64
CpuTime                    int64
ProcessCpuLoad           float64
CurrentThreadCpuTime       int64
CurrentThreaduserTime      int64
SystemLoadAverage        float64
TimeDiff                 float64
Outcome                   object
dtype: object

In [273]:
df.LogOp.unique() 
print(df.head())
print('----------------------------------------------------------')
print(df.info())

       EpochTime LogOp  freeMemory      CpuTime  ProcessCpuLoad  \
0  1596127227063  INFO   643836128  65660059000        0.062500   
1  1596127227067  INFO   643836128  65667444000        0.133297   
2  1596127227067  INFO   643836128  65667509000        0.064619   
3  1596127227067  INFO   643836128  65667551000        0.062500   
4  1596127227067  INFO   643836128  65667599000        0.063802   

   CurrentThreadCpuTime  CurrentThreaduserTime  SystemLoadAverage  TimeDiff  \
0            5940593000             4914928000           3.660156       0.0   
1            5942996000             4915261000           3.660156       4.0   
2            5943055000             4915299000           3.660156       0.0   
3            5943095000             4915322000           3.660156       0.0   
4            5943143000             4915346000           3.660156       0.0   

  Outcome  
0      30  
1       0  
2       0  
3       0  
4       0  
--------------------------------------------------

In [13]:
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()
df = pd.get_dummies(df, columns=['LogOp'], prefix = ['LogOp'])
#scalar_list = ["EpochTime","freeMemory","CpuTime","CurrentThreadCpuTime","CurrentThreaduserTime","SystemLoadAverage"," SystemLoadAverage"," TimeDiff"]

In [14]:
min_max_scaler = preprocessing.MinMaxScaler()

# for col in scalar_list:
df[["EpochTime","freeMemory","CpuTime","CurrentThreadCpuTime","CurrentThreaduserTime","SystemLoadAverage","TimeDiff","Outcome"]] = min_max_scaler.fit_transform(df[["EpochTime","freeMemory","CpuTime","CurrentThreadCpuTime","CurrentThreaduserTime","SystemLoadAverage","TimeDiff","Outcome"]])


In [15]:
df.head()

,EpochTime,freeMemory,CpuTime,ProcessCpuLoad,CurrentThreadCpuTime,CurrentThreaduserTime,SystemLoadAverage,TimeDiff,Outcome,LogOp_ERROR,LogOp_INFO,LogOp_OUTAGE,LogOp_WARN
0,0.896938,0.420325,0.297998,0.062500,0.606610,0.621827,0.491315,0.000000,0.3,0,1,0,0
1,0.896938,0.420325,0.298032,0.133297,0.606855,0.621869,0.491315,0.001405,0.0,0,1,0,0
2,0.896938,0.420325,0.298032,0.064619,0.606861,0.621874,0.491315,0.000000,0.0,0,1,0,0
3,0.896938,0.420325,0.298032,0.062500,0.606865,0.621877,0.491315,0.000000,0.0,0,1,0,0
4,0.896938,0.420325,0.298033,0.063802,0.606870,0.621880,0.491315,0.000000,0.0,0,1,0,0


In [16]:
df.Outcome.unique()

array([0.3, 0. , 0.9, 1. , 0.6])

In [24]:
import xgboost as xgb

In [25]:
df.dtypes

EpochTime                float64
freeMemory               float64
CpuTime                  float64
ProcessCpuLoad           float64
CurrentThreadCpuTime     float64
CurrentThreaduserTime    float64
SystemLoadAverage        float64
TimeDiff                 float64
Outcome                  float64
LogOp_ERROR                uint8
LogOp_INFO                 uint8
LogOp_OUTAGE               uint8
LogOp_WARN                 uint8
dtype: object

In [26]:
x = df.drop('Outcome', axis = 1)
x[0:5]

,EpochTime,freeMemory,CpuTime,ProcessCpuLoad,CurrentThreadCpuTime,CurrentThreaduserTime,SystemLoadAverage,TimeDiff,LogOp_ERROR,LogOp_INFO,LogOp_OUTAGE,LogOp_WARN
0,0.896938,0.420325,0.297998,0.062500,0.606610,0.621827,0.491315,0.000000,0,1,0,0
1,0.896938,0.420325,0.298032,0.133297,0.606855,0.621869,0.491315,0.001405,0,1,0,0
2,0.896938,0.420325,0.298032,0.064619,0.606861,0.621874,0.491315,0.000000,0,1,0,0
3,0.896938,0.420325,0.298032,0.062500,0.606865,0.621877,0.491315,0.000000,0,1,0,0
4,0.896938,0.420325,0.298033,0.063802,0.606870,0.621880,0.491315,0.000000,0,1,0,0


In [27]:
y = df[["Outcome"]]
y[0:5]

,Outcome
0,0.3
1,0.0
2,0.0
3,0.0
4,0.0


In [54]:
x_trainset, x_testset, y_trainset, y_testset = train_test_split(x,y,stratify=y, test_size=0.2, random_state=3)

In [55]:
model1 = xgb.XGBRegressor(objective = 'reg:linear', learning_rate=0.8,
                           alpha=10, subsample=0.8, colsample_bytree=1, n_estimators = 10, max_depth = 5)
model1.fit(x_trainset,y_trainset)


[13:29:25] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[13:29:25] WARNING: /Users/travis/build/dmlc/xgboost/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.8, max_delta_step=0, max_depth=5,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=10, n_jobs=0, num_parallel_tree=1,
             objective='reg:linear', random_state=0, reg_alpha=10, reg_lambda=1,
             scale_pos_weight=1, subsample=0.8, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [56]:
import numpy as np
from sklearn.metrics import mean_squared_error

In [57]:
y_test_pred = model1.predict(x_testset)
rmse = np.sqrt(mean_squared_error(y_testset, y_test_pred))
print("RMSE: %f" % (rmse))

RMSE: 0.005621


In [59]:
params = {"objective":"reg:linear",'colsample_bytree': 0.3,'learning_rate': 0.1,
                'max_depth': 5, 'alpha': 10}


In [62]:
import pickle

In [63]:
pickle.dump(model1,open('xgb_model.pkl','wb'))

In [69]:
from sklearn import linear_model
from sklearn.metrics import r2_score
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [66]:
df.head()

,EpochTime,freeMemory,CpuTime,ProcessCpuLoad,CurrentThreadCpuTime,CurrentThreaduserTime,SystemLoadAverage,TimeDiff,Outcome,LogOp_ERROR,LogOp_INFO,LogOp_OUTAGE,LogOp_WARN
0,0.896938,0.420325,0.297998,0.062500,0.606610,0.621827,0.491315,0.000000,0.3,0,1,0,0
1,0.896938,0.420325,0.298032,0.133297,0.606855,0.621869,0.491315,0.001405,0.0,0,1,0,0
2,0.896938,0.420325,0.298032,0.064619,0.606861,0.621874,0.491315,0.000000,0.0,0,1,0,0
3,0.896938,0.420325,0.298032,0.062500,0.606865,0.621877,0.491315,0.000000,0.0,0,1,0,0
4,0.896938,0.420325,0.298033,0.063802,0.606870,0.621880,0.491315,0.000000,0.0,0,1,0,0


In [70]:
reg = linear_model.LinearRegression()
reg.fit(x_trainset,y_trainset)

LinearRegression()

In [71]:
y_test_pred_reg = reg.predict(x_testset)

In [73]:
rmse_reg = np.sqrt(mean_squared_error(y_testset, y_test_pred_reg))

In [74]:
print("RMSE: %f" % (rmse_reg))

RMSE: 0.003674


In [76]:
from statsmodels.api import OLS
OLS(y_trainset,x_trainset).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                Outcome   R-squared:                       0.989
Model:                            OLS   Adj. R-squared:                  0.989
Method:                 Least Squares   F-statistic:                 1.263e+06
Date:                Sat, 01 Aug 2020   Prob (F-statistic):               0.00
Time:                        19:41:34   Log-Likelihood:             6.7405e+05
No. Observations:              156956   AIC:                        -1.348e+06
Df Residuals:                  156944   BIC:                        -1.348e+06
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
EpochTime                -0.0004    7.2e-05     -5.760      0.000      -0.001      -0.000
freeMemory                0.0001   4.63e-05      2.190      0.029    1.06e-05       0.000
CpuTime                   0.0024      0.000      7.933      0.000       0.002       0.003
ProcessCpuLoad         -4.67e-06   1.67e-05     -0.280      0.779   -3.73e-05     2.8e-05
CurrentThreadCpuTime     -0.0054      0.001     -7.170      0.000      -0.007      -0.004
CurrentThreaduserTime     0.0052      0.001      6.814      0.000       0.004       0.007
SystemLoadAverage         0.0002   6.37e-05      2.589      0.010    4.01e-05       0.000
TimeDiff                  0.3765      0.002    221.358      0.000       0.373       0.380
LogOp_ERROR               0.8732      0.000   2838.765      0.000       0.873       0.874
LogOp_INFO               -0.0003   4.91e-05     -6.929      0.000      -0.000      -0.000
LogOp_OUTAGE              1.0003      0.001   1003.783      0.000       0.998       1.002
LogOp_WARN               -0.0003   5.04e-05     -5.589      0.000      -0.000      -0.000
==============================================================================
Omnibus:                   482633.279   Durbin-Watson:                   2.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):     264283128864.243
Skew:                          46.181   Prob(JB):                         0.00
Kurtosis:                    6359.317   Cond. No.                         350.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""